In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import time
import csv
from sklearn.inspection import permutation_importance

In [63]:
def find_important_feature(model,file_numb):
    df = pd.read_csv(r'C:\Users\johnk\OneDrive\Desktop\project\python project\kaggle\amex-default-prediction\new_train\train_{0}.csv'.format(7))
    value_counts = pd.DataFrame(list(df.describe().iloc[0]), columns = ['counts'])
    col = pd.DataFrame(df.describe().iloc[0].index, columns = ['col'])
    df_count = pd.concat([col,value_counts], axis=1)
    keep = df_count.loc[df_count['counts']>1000].iloc[:-1]
    feature_name = list(keep['col'])
    label = pd.DataFrame(df['target'], columns=['target'])
    np_label =  np.array(label).T[0]
    for y in feature_name:
        df[y]=df[y].fillna(df[y].mean())
    feature = df[feature_name]
    if model == LogisticRegression:
        mdl = model(max_iter =20**9)
    else: 
        mdl = model()
    mdl.fit(feature, np_label)
    feature_importance = permutation_importance(mdl, feature, np_label, n_repeats=10)
    fi = pd.DataFrame(feature_importance.importances_mean, index = feature.columns, columns = ['importance'])
    important_l1= fi.loc[fi['importance']>0].index
    return important_l1

In [3]:
def update_important_l1(model, file_number):
    i = pd.read_csv('important_l1.csv')
    important_l1 = find_important_feature(model, file_number)
    if model == GradientBoostingClassifier:
        for x in important_l1:
            i.loc[0,x]+=1
    if model == LogisticRegression:
        for x in important_l1:
            i.loc[1,x]+=1
    i.to_csv('important_l1.csv', index = False)

In [65]:
models = [GradientBoostingClassifier]

In [ ]:
for x in range(0,15):
    start_time = time.time()
    for m in models:
        important_l1 = update_important_l1(m,x)
        print('. ', end='')
    print(x, end=' ')
    print(" %s second " %(time.time()-start_time))